# AWS Glue Studio Notebook
## Data Engineer Assesment: Identify potential Customers for new credit card


#### Optional: Run this cell to see available notebook commands ("magics").


####  Run this cell to set up and start your interactive session.


In [1]:
%idle_timeout 5880
%glue_version 5.0
%worker_type G.1X
%number_of_workers 5
%additional_python_modules scikit-learn

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import calendar
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.7 
Current idle_timeout is None minutes.
idle_timeout has been set to 5880 minutes.
Setting Glue version to: 5.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Additional python modules to be included:
scikit-learn
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Idle Timeout: 5880
Session ID: 292aea95-0f24-4218-8839-10dcad14b181
Applying the following default arguments:
--glue_kernel_version 1.0.7
--enable-glue-datacatalog true
--additional-python-modules scikit-learn
Waiting for session 292aea95-0f24-42

#### Create a DynamicFrame from a table:test_data in the AWS Glue Data Catalog and display its schema


In [2]:
dyf = glueContext.create_dynamic_frame.from_catalog(database='assessmnt_f_n_b', table_name='train_data')
dyf.printSchema()

root
|-- customerid: long
|-- sex: string
|-- ownsvehicle: string
|-- ownsproperty: string
|-- numchildren: long
|-- annualincome: double
|-- jobstatus: string
|-- educationlevel: string
|-- relationshipstatus: string
|-- livingarrangement: string
|-- customerage: long
|-- jobduration: long
|-- mobilephone: long
|-- workphone: long
|-- homephone: long
|-- email: long
|-- occupation: string
|-- householdsize: double
|-- accountduration: double
|-- creditrisk: long


In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 

#### Convert the DynamicFrame to a Spark DataFrame and display a sample of the data


In [5]:
df = dyf.toDF()
df.show()

+----------+---+-----------+------------+-----------+------------+--------------------+--------------------+--------------------+-------------------+-----------+-----------+-----------+---------+---------+-----+--------------------+-------------+---------------+----------+
|customerid|sex|ownsvehicle|ownsproperty|numchildren|annualincome|           jobstatus|      educationlevel|  relationshipstatus|  livingarrangement|customerage|jobduration|mobilephone|workphone|homephone|email|          occupation|householdsize|accountduration|creditrisk|
+----------+---+-----------+------------+-----------+------------+--------------------+--------------------+--------------------+-------------------+-----------+-----------+-----------+---------+---------+-----+--------------------+-------------+---------------+----------+
|   5037048|  M|          Y|           Y|          0|    135000.0|             Working|Secondary / secon...|             Married|       With parents|     -16271|      -3111|     

### Testing SQL query

In [6]:
df = dyf.toDF()
df.registerTempTable('train_data')

glueContext.sql("""
select * from train_data limit 10
""").show()



+----------+---+-----------+------------+-----------+------------+--------------------+--------------------+--------------------+-------------------+-----------+-----------+-----------+---------+---------+-----+-----------+-------------+---------------+----------+
|customerid|sex|ownsvehicle|ownsproperty|numchildren|annualincome|           jobstatus|      educationlevel|  relationshipstatus|  livingarrangement|customerage|jobduration|mobilephone|workphone|homephone|email| occupation|householdsize|accountduration|creditrisk|
+----------+---+-----------+------------+-----------+------------+--------------------+--------------------+--------------------+-------------------+-----------+-----------+-----------+---------+---------+-----+-----------+-------------+---------------+----------+
|   5037048|  M|          Y|           Y|          0|    135000.0|             Working|Secondary / secon...|             Married|       With parents|     -16271|      -3111|          1|        0|        0|

## Understanding our data/ EDA non-graphical

In [7]:
df2 = df.toPandas()
df2.info() 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29165 entries, 0 to 29164
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   customerid          29165 non-null  int64  
 1   sex                 29165 non-null  object 
 2   ownsvehicle         29165 non-null  object 
 3   ownsproperty        29165 non-null  object 
 4   numchildren         29165 non-null  int64  
 5   annualincome        29165 non-null  float64
 6   jobstatus           29165 non-null  object 
 7   educationlevel      29165 non-null  object 
 8   relationshipstatus  29165 non-null  object 
 9   livingarrangement   29165 non-null  object 
 10  customerage         29165 non-null  int64  
 11  jobduration         29165 non-null  int64  
 12  mobilephone         29165 non-null  int64  
 13  workphone           29165 non-null  int64  
 14  homephone           29165 non-null  int64  
 15  email               29165 non-null  int64  
 16  occu

In [8]:
df2.shape

(29165, 20)


In [13]:
describe = df2.describe(include='object')
print(describe)

          sex ownsvehicle ownsproperty jobstatus  \
count   29165       29165        29165     29165   
unique      2           2            2         5   
top         F           N            Y   Working   
freq    19549       18128        19557     15056   

                       educationlevel relationshipstatus  livingarrangement  \
count                           29165              29165              29165   
unique                              5                  5                  6   
top     Secondary / secondary special            Married  House / apartment   
freq                            19803              20044              26059   

       occupation  
count       29165  
unique         19  
top                
freq         9027


In [14]:
describe = df2.describe(include='all')
print(describe)

          customerid    sex ownsvehicle ownsproperty   numchildren  \
count   2.916500e+04  29165       29165        29165  29165.000000   
unique           NaN      2           2            2           NaN   
top              NaN      F           N            Y           NaN   
freq             NaN  19549       18128        19557           NaN   
mean    5.078232e+06    NaN         NaN          NaN      0.430790   
std     4.182400e+04    NaN         NaN          NaN      0.741882   
min     5.008804e+06    NaN         NaN          NaN      0.000000   
25%     5.042047e+06    NaN         NaN          NaN      0.000000   
50%     5.074666e+06    NaN         NaN          NaN      0.000000   
75%     5.114629e+06    NaN         NaN          NaN      1.000000   
max     5.150485e+06    NaN         NaN          NaN     19.000000   

        annualincome jobstatus                 educationlevel  \
count   2.916500e+04     29165                          29165   
unique           NaN         

In [11]:
df2.describe()


         customerid   numchildren  ...  accountduration    creditrisk
count  2.916500e+04  29165.000000  ...     29165.000000  29165.000000
mean   5.078232e+06      0.430790  ...       -26.137734      0.017110
std    4.182400e+04      0.741882  ...        16.486702      0.129682
min    5.008804e+06      0.000000  ...       -60.000000      0.000000
25%    5.042047e+06      0.000000  ...       -39.000000      0.000000
50%    5.074666e+06      0.000000  ...       -24.000000      0.000000
75%    5.114629e+06      1.000000  ...       -12.000000      0.000000
max    5.150485e+06     19.000000  ...         0.000000      1.000000

[8 rows x 12 columns]


In [15]:
#checking the null values
total = df2.isnull().sum().sort_values(ascending=False)
percent = (df2.isnull().sum()/df2.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

                    Total  Percent
customerid              0      0.0
sex                     0      0.0
ownsvehicle             0      0.0
ownsproperty            0      0.0
numchildren             0      0.0
annualincome            0      0.0
jobstatus               0      0.0
educationlevel          0      0.0
relationshipstatus      0      0.0
livingarrangement       0      0.0
customerage             0      0.0
jobduration             0      0.0
mobilephone             0      0.0
workphone               0      0.0
homephone               0      0.0
email                   0      0.0
occupation              0      0.0
householdsize           0      0.0
accountduration         0      0.0
creditrisk              0      0.0


In [9]:
%%sql
select * from assessmnt_f_n_b.train_data where occupation = '' limit 10;

+----------+---+-----------+------------+-----------+------------+--------------------+--------------------+--------------------+-------------------+-----------+-----------+-----------+---------+---------+-----+----------+-------------+---------------+----------+
|customerid|sex|ownsvehicle|ownsproperty|numchildren|annualincome|           jobstatus|      educationlevel|  relationshipstatus|  livingarrangement|customerage|jobduration|mobilephone|workphone|homephone|email|occupation|householdsize|accountduration|creditrisk|
+----------+---+-----------+------------+-----------+------------+--------------------+--------------------+--------------------+-------------------+-----------+-----------+-----------+---------+---------+-----+----------+-------------+---------------+----------+
|   5105858|  F|          N|           N|          0|    270000.0|             Working|Secondary / secon...|           Separated|  House / apartment|     -16207|       -515|          1|        0|        1|   

### Handling empty values in occupation column, I replaced emty values with string 'Unknown'

In [10]:
df3 = df2.copy()

df3.loc[df3['occupation'].str.len() == 0, 'occupation'] = 'Unknown'

In [11]:
#checkthere is stll empty values
empty_count = df2['occupation'].isin(['', None]).sum()
print(f"Number of empty or NaN values: {empty_count}")

Number of empty or NaN values: 9027


In [12]:
#checkthere is stll empty values
empty_count = df3['occupation'].isin(['', None]).sum()
print(f"Number of empty or NaN values: {empty_count}")

Number of empty or NaN values: 0


In [13]:
print(df3['occupation'].unique())

['Core staff' 'Accountants' 'Laborers' 'Managers' 'Unknown' 'Sales staff'
 'Medicine staff' 'High skill tech staff' 'HR staff' 'Low-skill Laborers'
 'Drivers' 'Secretaries' 'Cleaning staff' 'Cooking staff' 'Security staff'
 'Private service staff' 'IT staff' 'Waiters/barmen staff' 'Realty agents']


## removing columns I do not need

In [14]:
df3=df3.drop(['customerid','customerage','jobduration','accountduration'], axis=1)

In [13]:
print(df3.sample(10))

      sex ownsvehicle ownsproperty  numchildren  annualincome  \
28500   M           Y            Y            0      202500.0   
14322   F           N            N            1      159750.0   
20815   M           Y            N            0      225000.0   
1823    F           N            Y            0      225000.0   
19188   F           N            Y            0       72000.0   
17660   F           N            N            0      292500.0   
23860   F           N            Y            0       90000.0   
26946   M           N            N            0      180000.0   
21980   M           Y            N            1      135000.0   
10658   M           Y            Y            2      315000.0   

                  jobstatus                 educationlevel relationshipstatus  \
28500  Commercial associate  Secondary / secondary special            Married   
14322  Commercial associate               Higher education     Civil marriage   
20815  Commercial associate  Secondary / 

In [15]:
df3.shape

(29165, 16)


## encoding

In [16]:
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df3.sex = le.fit_transform(df3.sex)
df3.ownsvehicle = le.fit_transform(df3.ownsvehicle)
df3.ownsproperty = le.fit_transform(df3.ownsproperty)
df3.jobstatus = le.fit_transform(df3.jobstatus)
df3.educationlevel = le.fit_transform(df3.educationlevel)
df3.relationshipstatus= le.fit_transform(df3.relationshipstatus)
df3.livingarrangement= le.fit_transform(df3.livingarrangement)
df3.mobilephone = le.fit_transform(df3.mobilephone)
df3.workphone= le.fit_transform(df3.workphone)
df3.homephone= le.fit_transform(df3.homephone)
df3.email= le.fit_transform(df3.email)
df3.occupation= le.fit_transform(df3.occupation)

###  imbalanced datasets pose a significant challenge because standard classification algorithms tend to be biased towards the majority class
#### "creditrisk" column is my target variable, I have to check its value distribution

In [17]:
#Checking the number of fraud in percentage
#  I willn name those who are not credit risk "safe" and those who are credit risk "fradulent"
count_1 = df3[df3["creditrisk"] == 1].value_counts().sum()
totalResponse = df3["creditrisk"].value_counts().sum()
print("The percentage of customers who are creditrisk  classification in train data is :", round(count_1*100/totalResponse),"%")

The percentage of customers who are creditrisk  classification in train data is : 2 %


In [18]:
count_1 = df3[df3["creditrisk"] == 0].value_counts().sum()
totalResponse = df3["creditrisk"].value_counts().sum()
print("The percentage of customers who are not creditrisk  classification in train data is :", round(count_1*100/totalResponse),"%")

The percentage of customers who are not creditrisk  classification in train data is : 98 %


## Embalancing

In [19]:
from sklearn.utils import resample

In [20]:
#from sklearn.utils import resample
safe = df3[df3['creditrisk']== 0]
fradulent = df3[df3['creditrisk']== 1]
saf = resample(safe,
                          replace=True, 
                          n_samples=100000, # match number in minority class
                          random_state=27) # reproducible results


fraud = resample(fradulent,
                          replace=True, # sample without replacement (no need to duplicate observations)
                          n_samples=100000, # match number in minority class
                          random_state=27) # reproducible results

# Combine downsampled majority class with minority class
df3 = pd.concat([saf,fraud])

In [21]:
# tells how much a data distribution deviates from a perfectly symmetrical bell curve (normal distribution).
df3.skew()


sex                   0.600191
ownsvehicle           0.528989
ownsproperty         -0.517702
numchildren           2.075812
annualincome          2.154850
jobstatus            -0.266312
educationlevel       -0.838375
relationshipstatus    1.149909
livingarrangement     3.297323
mobilephone           0.000000
workphone             1.275419
homephone             0.896543
email                 2.960718
occupation           -0.177882
householdsize         1.078569
creditrisk            0.000000
dtype: float64


In [22]:
# kurtosis measures how much a distribution's tails deviate from the tails of a normal distribution. Checks in all columns
df3.kurtosis()

sex                   -1.639787
ownsvehicle           -1.720187
ownsproperty          -1.732002
numchildren           10.701421
annualincome          10.025005
jobstatus             -1.727383
educationlevel        -1.228742
relationshipstatus     0.364238
livingarrangement      9.680967
mobilephone            0.000000
workphone             -0.373311
homephone             -1.196223
email                  6.765918
occupation            -1.373279
householdsize          3.836886
creditrisk            -2.000020
dtype: float64


### Train test

In [23]:
# Packages for training models
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
#import xgboost as xgb

# Model Evaluation Packages
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.metrics import make_scorer

In [24]:
y = df3['creditrisk']
X = df3.drop('creditrisk', axis=1)

## scalling

In [25]:
# scalling the Data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), index=X.index, columns=X.columns)
        

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


In [27]:
#values after downsample and upsample
y_train.value_counts(normalize=True)*100

creditrisk
0    50.004375
1    49.995625
Name: proportion, dtype: float64


## Decision tres

In [28]:
rf= RandomForestClassifier(random_state=1)
rf.fit(X_train, y_train)
pred1 = rf.predict(X_test)

lr = LogisticRegression()
lr.fit(X_train, y_train)
pred2 = lr.predict(X_test)

kn=KNeighborsClassifier(n_neighbors=5)
kn.fit(X_train, y_train)
pred3 = kn.predict(X_test)

dt=DecisionTreeClassifier()
dt.fit(X_train, y_train)
pred4 = dt.predict(X_test)

## Random Forest

In [29]:
predictions = rf.predict(X_test)
predictions

array([1, 0, 1, ..., 1, 1, 0])


In [34]:
print(classification_report(y_test, pred1))

              precision    recall  f1-score   support

           0       1.00      0.95      0.97     19993
           1       0.95      1.00      0.97     20007

    accuracy                           0.97     40000
   macro avg       0.98      0.97      0.97     40000
weighted avg       0.98      0.97      0.97     40000


## Logistic Regression

In [30]:
predictions = lr.predict(X_test)
predictions

array([0, 0, 0, ..., 1, 1, 0])


In [31]:
print(classification_report(y_test, pred2))

              precision    recall  f1-score   support

           0       0.55      0.58      0.56     19993
           1       0.55      0.52      0.53     20007

    accuracy                           0.55     40000
   macro avg       0.55      0.55      0.55     40000
weighted avg       0.55      0.55      0.55     40000


## KNeighbors Classifier

In [32]:
predictions = kn.predict(X_test)
predictions

array([1, 0, 1, ..., 1, 1, 0])


## DecisionTree Classifier

In [33]:
predictions = dt.predict(X_test)
predictions

array([1, 0, 1, ..., 1, 1, 0])


In [34]:
print(classification_report(y_test, pred4))

              precision    recall  f1-score   support

           0       1.00      0.95      0.97     19993
           1       0.95      1.00      0.97     20007

    accuracy                           0.97     40000
   macro avg       0.98      0.97      0.97     40000
weighted avg       0.98      0.97      0.97     40000


## K- fold validation

In [35]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold = KFold(5)

In [36]:
# RandomForest

results=cross_val_score(rf,X,y,cv=kfold)
print(results*100,'\n')

print(np.mean(results)*100)

[94.875  94.9575 97.48   98.965  99.23  ] 

97.1015


In [37]:
# Logistic Regression

results=cross_val_score(lr,X,y,cv=kfold)
print(results*100,'\n')

print(np.mean(results)*100)

[ 3.13    3.3325 54.535   9.7775  9.56  ] 

16.067000000000004


In [38]:
# Decision tree Regression

results=cross_val_score(dt,X,y,cv=kfold)
print(results*100,'\n')

print(np.mean(results)*100)

[94.7875 94.93   97.4725 98.965  99.23  ] 

97.07700000000001


In [39]:
import pickle

model_save_path = "tmodel94.pkl"
with open(model_save_path,'wb') as file:
    pickle.dump(rf,file)